In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
import torch.utils as utils
import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import torchvision.datasets as dsets
from torchvision import models
from torchvision.utils import save_image
#from torch.utils.tensorboard import SummaryWriter

#writer = SummaryWriter('runs/thesis06_miniIN5w1s_5step')

from tqdm import tqdm

from flows import PlanarFlow
from utils import Binarize
from codes import Linear_flipout, Flatten, count_parameters, EfficientNet

from torchmeta.datasets import MiniImagenet
from torchmeta.transforms import Categorical, ClassSplitter, Rotation
from torchvision.transforms import Compose, Resize, ToTensor
from torchmeta.utils.data import BatchMetaDataLoader


#from __future__ import print_function
import argparse
import cv2
import matplotlib.pyplot as plt

import os
cur_dir = "C:/Users/KJH/OneDrive - skku.edu/KJH/Projects/2019winter_research"
#cur_dir = "C:/Users/KJH-Laptop/OneDrive - skku.edu/KJH/Projects/2019winter_research/"
os.chdir(cur_dir)
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

import time
import copy
import random as rd

device = torch.device('cuda')

class net(nn.Module):
    def __init__(self, num_classes):
        super(net, self).__init__()
        self.input_dim = [3, 84, 84]
        self.num_classes = num_classes
        
        self.ctx = torch.hub.load('rwightman/gen-efficientnet-pytorch', 'efficientnet_b0', pretrained=True)
        self.ctx.classifier = nn.Identity()
        
        self.layer = nn.LSTM(input_size=1280, hidden_size=1280, num_layers=5, bias=True, batch_first = True).to(device)
        self.dec_mu = nn.ModuleList()
        self.dec_logvar = nn.ModuleList()
        
        self.num_params = [[200 * 200, 200], [200 * 200, 200], [200 * self.num_classes, self.num_classes]]
        
        for layer_size in self.num_params:
            self.dec_mu.append(
                nn.Sequential(
                    nn.ELU(),
                    nn.Linear(1280, 1280, bias = True),
                    nn.ELU(),
                    nn.Linear(1280, layer_size[0] + layer_size[1], bias = True),
                ).to(device))
            
            self.dec_logvar.append(
                nn.Sequential(
                    nn.ELU(),
                    nn.Linear(1280, 1280, bias = True),
                    nn.ELU(),
                    nn.Linear(1280, layer_size[0] + layer_size[1], bias = True),
                ).to(device))
        
        self.encoder = nn.Sequential(
            torch.hub.load('rwightman/gen-efficientnet-pytorch', 'efficientnet_b0', pretrained=True),
            Flatten(),            
            nn.Linear(1280, 200, bias = False),
            nn.ELU()
        )
        self.encoder[0].classifier = nn.Identity()
            
            
    def forward(self, input_train, label_train, input_test, label_test, adapt_lr, adapt_step = 1):
        ctx = self.ctx(input_train).view(input_train.shape[0], self.num_classes, 256)
        h = torch.stack([ctx[torch.where(label_train == x)].mean(dim = 0) for x in range(self.num_classes)], dim = 1)
        h = h.view(self.num_classes, 1, -1)
        
        params = nn.ParameterList()
        param_vals = []
        kld = torch.tensor(0., device=device)
        c = torch.empty([5, 1, 1280], requires_grad = False, device = device).fill_(0)
        x_train_init = self.encoder(input_train)
        x_train = x_train_init

        for ind, (dec_mu, dec_logvar) in enumerate(zip(self.dec_mu, self.dec_logvar)):
            x = torch.empty([1, 1, 1280], requires_grad=False, device = device).normal_(0, 1)
            f, (h, c) = self.layer(x, (h, c))
            
            mu = dec_mu(f.view(1, -1)).squeeze()
            logvar = dec_logvar(f.view(1, -1)).squeeze()
            params.append(nn.Parameter(torch.stack((mu, logvar), dim = 0), requires_grad = True))
        
        optimizer = optim.SGD(params, lr = adapt_lr)
        
        for step in range(adapt_step):
            x_train = x_train_init
            kld = torch.tensor(0., device=device)
            for ind, param in enumerate(params):
                param.retain_grad()

                weight_mu = param[0, :self.num_params[ind][0]].view(-1, self.num_params[ind][1])
                bias_mu = param[0, self.num_params[ind][0]:]

                weight_logvar = param[1, :self.num_params[ind][0]].view(-1, self.num_params[ind][1])
                bias_logvar = param[1, self.num_params[ind][0]:]

                weight_noise = torch.empty(weight_mu.shape, requires_grad = False, device = device).normal_(0,1)
                bias_noise = torch.empty(bias_mu.shape, requires_grad = False, device = device).normal_(0,1)
                in_sign = torch.empty(x_train.shape, requires_grad = False, device = device).uniform_(-1,1).sign()
                out_sign = torch.empty([x_train.shape[0], self.num_params[ind][1]], requires_grad = False, device = device).uniform_(-1,1).sign()

                x_train = torch.mm(x_train, weight_mu) + torch.mm(in_sign * x_train, weight_noise * weight_mu * weight_logvar.div(2).exp()) * out_sign
                x_train += (1 + bias_noise * bias_logvar.div(2).exp()) * bias_mu
                x_train = F.elu(x_train)

                kld += (mu.pow(2) - logvar + logvar.exp() - 1).mean()/2
                
            if step < adapt_step:
                optimizer.zero_grad()
                loss = F.cross_entropy(x_train, label_train) + 1e-6 * kld
                loss.backward(retain_graph = True)
                optimizer.step()
                
                
        x_test_init = self.encoder(input_test)
        x_test = x_test_init
        kld = torch.tensor(0., device=device)
        for ind, param in enumerate(params):
            param.retain_grad()

            weight_mu = param[0, :self.num_params[ind][0]].view(-1, self.num_params[ind][1])
            bias_mu = param[0, self.num_params[ind][0]:]

            weight_logvar = param[1, :self.num_params[ind][0]].view(-1, self.num_params[ind][1])
            bias_logvar = param[1, self.num_params[ind][0]:]

            weight_noise = torch.empty(weight_mu.shape, requires_grad = False, device = device).normal_(0,1)
            bias_noise = torch.empty(bias_mu.shape, requires_grad = False, device = device).normal_(0,1)
            in_sign = torch.empty(x_test.shape, requires_grad = False, device = device).uniform_(-1,1).sign()
            out_sign = torch.empty([x_test.shape[0], self.num_params[ind][1]], requires_grad = False, device = device).uniform_(-1,1).sign()

            x_test = torch.mm(x_test, weight_mu) + torch.mm(in_sign * x_test, weight_noise * weight_mu * weight_logvar.div(2).exp()) * out_sign
            x_test += (1 + bias_noise * bias_logvar.div(2).exp()) * bias_mu
            x_test = F.elu(x_test)

            kld += (mu.pow(2) - logvar + logvar.exp() - 1).mean()/2

        return x_test, kld

In [2]:
batch_size = 16
meta_trainset = MiniImagenet('./data/',
                   # Number of ways
                   num_classes_per_task=5,
                   # Resize the images to 28x28 and converts them to PyTorch tensors (from Torchvision)
                   transform=Compose([Resize(84), ToTensor()]), #Compose([Resize(28), ToTensor()]),
                   # Transform the labels to integers (e.g. ("Glagolitic/character01", "Sanskrit/character14", ...) to (0, 1, ...))
                   target_transform=Categorical(num_classes=5),
                   # Creates new virtual classes with rotated versions of the images (from Santoro et al., 2016)
                   class_augmentations=[Rotation([90, 180, 270])],
                   meta_train=True,
                   download=True)
meta_trainset = ClassSplitter(meta_trainset, shuffle=True, num_train_per_class=1, num_test_per_class=15)
meta_trainloader = BatchMetaDataLoader(meta_trainset, batch_size=batch_size, num_workers=0)

model = net(5).cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

num_batches = 200
for batch_idx, meta_train_batch in zip(range(num_batches), meta_trainloader):
    start = time.time()

    train_inputs, train_targets = [x.to(device) for x in meta_train_batch["train"]]
    test_inputs, test_targets = [x.to(device) for x in meta_train_batch["test"]]
    
    cum_loss = torch.tensor(0., device=device)
    accuracy = torch.tensor(0., device=device)
    reg = torch.tensor(0., device=device)

    for task_idx, (train_input, train_target, test_input, test_target) in enumerate(
        zip(train_inputs, train_targets, test_inputs, test_targets)):
        optimizer.zero_grad()
        pred, kld = model(train_input, train_target, test_input, 0.5, 5)
        loss = criterion(pred, test_target)
        (loss + 1e-6 * kld).backward()
        optimizer.step()
        with torch.no_grad():
            cum_loss += loss
            accuracy += torch.sum(pred.argmax(1) == test_target.cuda())
            reg += kld
    cum_loss /= batch_size
    accuracy /= batch_size * 75
    reg /= batch_size

    if batch_idx % 10 == 0:
        print("%3d) loss = %f, kld = %f, acc = %f, time = %.3f sec" %(batch_idx, cum_loss, reg, accuracy, time.time() - start))

Using cache found in C:\Users\KJH/.cache\torch\hub\rwightman_gen-efficientnet-pytorch_master
Using cache found in C:\Users\KJH/.cache\torch\hub\rwightman_gen-efficientnet-pytorch_master


  0) loss = 1.605355, kld = 0.000905, acc = 0.222500, time = 15.218 sec
 10) loss = 1.584348, kld = 0.000673, acc = 0.280000, time = 14.296 sec
 20) loss = 1.583418, kld = 0.000526, acc = 0.270833, time = 14.411 sec
 30) loss = 1.584720, kld = 0.000418, acc = 0.283333, time = 14.567 sec
 40) loss = 1.599697, kld = 0.000337, acc = 0.297500, time = 14.171 sec
 50) loss = 1.507016, kld = 0.000277, acc = 0.345000, time = 15.022 sec
 60) loss = 1.447802, kld = 0.000234, acc = 0.379167, time = 14.804 sec
 70) loss = 1.519290, kld = 0.000195, acc = 0.336667, time = 15.111 sec
 80) loss = 1.507907, kld = 0.000169, acc = 0.324167, time = 15.292 sec
 90) loss = 1.455187, kld = 0.000152, acc = 0.367500, time = 15.357 sec
100) loss = 1.461949, kld = 0.000137, acc = 0.357500, time = 14.935 sec
110) loss = 1.451432, kld = 0.000124, acc = 0.358333, time = 15.059 sec
120) loss = 1.408308, kld = 0.000113, acc = 0.391667, time = 19.109 sec
130) loss = 1.441451, kld = 0.000105, acc = 0.376667, time = 31.

In [3]:
optimizer = optim.Adam(model.parameters(), lr=1e-5)

num_batches = 200
for batch_idx, meta_train_batch in zip(range(num_batches), meta_trainloader):
    start = time.time()

    train_inputs, train_targets = [x.to(device) for x in meta_train_batch["train"]]
    test_inputs, test_targets = [x.to(device) for x in meta_train_batch["test"]]
    
    cum_loss = torch.tensor(0., device=device)
    accuracy = torch.tensor(0., device=device)
    reg = torch.tensor(0., device=device)

    for task_idx, (train_input, train_target, test_input, test_target) in enumerate(
        zip(train_inputs, train_targets, test_inputs, test_targets)):
        optimizer.zero_grad()
        pred, kld = model(train_input, train_target, test_input, 0.5, 5)
        loss = criterion(pred, test_target)
        (loss + 1e-6 * kld).backward()
        optimizer.step()
        with torch.no_grad():
            cum_loss += loss
            accuracy += torch.sum(pred.argmax(1) == test_target.cuda())
            reg += kld
    cum_loss /= batch_size
    accuracy /= batch_size * 75
    reg /= batch_size

    if batch_idx % 10 == 0:
        print("%3d) loss = %f, kld = %f, acc = %f, time = %.3f sec" %(batch_idx, cum_loss, reg, accuracy, time.time() - start))

  0) loss = 1.418599, kld = 0.000079, acc = 0.398333, time = 29.644 sec
 10) loss = 1.443808, kld = 0.000076, acc = 0.367500, time = 28.839 sec
 20) loss = 1.397698, kld = 0.000079, acc = 0.373333, time = 29.358 sec
 30) loss = 1.446883, kld = 0.000078, acc = 0.355833, time = 26.758 sec
 40) loss = 1.432125, kld = 0.000080, acc = 0.372500, time = 39.550 sec
 50) loss = 1.498632, kld = 0.000078, acc = 0.340833, time = 22.855 sec
 60) loss = 1.457449, kld = 0.000080, acc = 0.356667, time = 18.775 sec
 70) loss = 1.380359, kld = 0.000080, acc = 0.410833, time = 31.489 sec
 80) loss = 1.457868, kld = 0.000079, acc = 0.378333, time = 18.383 sec
 90) loss = 1.383330, kld = 0.000079, acc = 0.405000, time = 24.191 sec
100) loss = 1.455279, kld = 0.000076, acc = 0.353333, time = 21.663 sec
110) loss = 1.477859, kld = 0.000078, acc = 0.349167, time = 32.409 sec
120) loss = 1.454750, kld = 0.000076, acc = 0.350833, time = 30.050 sec
130) loss = 1.424488, kld = 0.000077, acc = 0.385833, time = 25.

In [10]:
optimizer = optim.Adam(model.parameters(), lr=1e-7)

num_batches = 200
for batch_idx, meta_train_batch in zip(range(num_batches), meta_trainloader):
    start = time.time()

    train_inputs, train_targets = [x.to(device) for x in meta_train_batch["train"]]
    test_inputs, test_targets = [x.to(device) for x in meta_train_batch["test"]]
    
    cum_loss = torch.tensor(0., device=device)
    accuracy = torch.tensor(0., device=device)
    reg = torch.tensor(0., device=device)

    for task_idx, (train_input, train_target, test_input, test_target) in enumerate(
        zip(train_inputs, train_targets, test_inputs, test_targets)):
        optimizer.zero_grad()
        pred, kld = model(train_input, train_target, test_input, 0.5, 5)
        loss = criterion(pred, test_target)
        (loss + 1e-6 * kld).backward()
        optimizer.step()
        with torch.no_grad():
            cum_loss += loss
            accuracy += torch.sum(pred.argmax(1) == test_target.cuda())
            reg += kld
    cum_loss /= batch_size
    accuracy /= batch_size * 75
    reg /= batch_size

    if batch_idx % 10 == 0:
        print("%3d) loss = %f, kld = %f, acc = %f, time = %.3f sec" %(batch_idx, cum_loss, reg, accuracy, time.time() - start))

  0) loss = 1.364889, kld = 0.000078, acc = 0.415833, time = 14.061 sec


In [13]:
meta_testset  = MiniImagenet('./data/',
                   # Number of ways
                   num_classes_per_task=5,
                   # Resize the images to 28x28 and converts them to PyTorch tensors (from Torchvision)
                   transform=Compose([Resize(84), ToTensor()]),
                   # Transform the labels to integers (e.g. ("Glagolitic/character01", "Sanskrit/character14", ...) to (0, 1, ...))
                   target_transform=Categorical(num_classes=5),
                   # Creates new virtual classes with rotated versions of the images (from Santoro et al., 2016)
                   class_augmentations=[Rotation([90, 180, 270])],
                   meta_test=True,
                   download=True)
meta_testset = ClassSplitter(meta_testset, shuffle=True, num_train_per_class=1, num_test_per_class=15)
meta_testloader = BatchMetaDataLoader(meta_testset, batch_size=batch_size, num_workers=0)

tot_loss = torch.tensor(0., device=device)
tot_acc = torch.tensor(0., device=device)
tot_reg = torch.tensor(0., device=device)

for batch_idx, meta_test_batch in zip(range(num_batches), meta_testloader):
    start = time.time()
    train_inputs, train_targets = [x.to(device) for x in meta_test_batch["train"]]
    test_inputs, test_targets = [x.to(device) for x in meta_test_batch["test"]]
    
    cum_loss = torch.tensor(0., device=device)
    accuracy = torch.tensor(0., device=device)
    reg = torch.tensor(0., device=device)

    for task_idx, (train_input, train_target, test_input, test_target) in enumerate(
        zip(train_inputs, train_targets, test_inputs, test_targets)):
        optimizer.zero_grad()
        pred, kld = model(train_input, train_target, test_input, 0.5, 5)
        with torch.no_grad():            
            loss = criterion(pred, test_target)
            cum_loss += loss
            accuracy += torch.sum(pred.argmax(1) == test_target.cuda())
            reg += kld

    tot_loss += cum_loss / batch_size
    tot_acc += accuracy  / (batch_size * 75)
    tot_reg += reg / batch_size
        
tot_loss /= num_batches
tot_acc /= num_batches
tot_reg /= num_batches
        
print("loss = %f, test_kld = %f, meta_test_acc = %f, time = %.3f sec" %(tot_loss, tot_reg, tot_acc, time.time() - start))

loss = 1.457098, test_kld = 0.000077, meta_test_acc = 0.358529, time = 6.496 sec


In [6]:
torch.save(model.state_dict(), "./save/thesis06_miniIN5w1s_5step")